# Signature Forgery Detection - Google Colab

This notebook trains CNN and Siamese Network models for signature forgery detection using Contrastive Loss.

## Features
- **CNN Model**: Binary classification for signature verification
- **Siamese Network**: Pair-based verification with Contrastive Loss
- **Data Augmentation**: Enhanced augmentation for better generalization
- **Model Export**: ONNX and OpenVINO export support

## Setup Instructions
1. Run all cells in order
2. Upload your dataset (genuine/ and forged/ folders)
3. Train models
4. Evaluate and visualize results


In [ ]:
# Install required packages
!pip install tensorflow>=2.10.0 numpy>=1.21.0 matplotlib>=3.5.0 opencv-python>=4.5.0 scikit-learn>=1.0.0 pillow>=9.0.0 seaborn>=0.11.0 tf2onnx>=1.14.0 -q

print("✅ All packages installed successfully!")


In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive, files
import zipfile
import shutil

# Mount Google Drive (optional - if you want to use data from Drive)
# Uncomment the line below if you want to use data from Google Drive
# drive.mount('/content/drive')

print("✅ Imports successful!")


## Step 1: Setup Project Structure

Create necessary directories for the project.


In [ ]:
# Create project structure
os.makedirs('src', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)
os.makedirs('data', exist_ok=True)

print("✅ Project structure created!")
print("📁 Directories: src/, models/, results/, data/")


In [ ]:
## Step 2: Clone Repository or Upload Source Files

**Option A: Clone from GitHub (Recommended)**


In [ ]:
# Clone repository from GitHub
!git clone https://github.com/rumeysaai/Signuture_Forgery_Detection.git temp_repo
!cp -r temp_repo/src/* /content/src/
!rm -rf temp_repo

# Add src to Python path
sys.path.insert(0, '/content/src')

print("✅ Repository cloned and source files copied!")
print("📂 Source files available in /content/src/")


**Option B: Manual Upload (If GitHub clone doesn't work)**

If the clone above doesn't work, manually upload these files:
- `src/models.py`
- `src/train.py`
- `src/utils.py`


In [ ]:
# Manual upload (uncomment if needed)
# from google.colab import files
# uploaded = files.upload()
# 
# # Move uploaded files to src/
# for filename in uploaded.keys():
#     if filename.endswith('.py'):
#         shutil.move(filename, f'src/{filename}')
#         print(f"Moved {filename} to src/")


## Step 3: Upload Dataset

Upload your dataset. The dataset should contain:
- `genuine/` folder with genuine signature images
- `forged/` folder with forged signature images

**Option A: Upload ZIP file (Recommended)**


In [ ]:
# Upload ZIP file containing genuine/ and forged/ folders
from google.colab import files
import zipfile

uploaded = files.upload()

# Extract if ZIP file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('data')
        print(f"✅ Extracted {filename} to data/")
        # Check structure
        if os.path.exists('data/genuine') and os.path.exists('data/forged'):
            genuine_count = len([f for f in os.listdir('data/genuine') if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            forged_count = len([f for f in os.listdir('data/forged') if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            print(f"📊 Found {genuine_count} genuine signatures")
            print(f"📊 Found {forged_count} forged signatures")
        else:
            print("⚠️ Warning: genuine/ or forged/ folders not found in ZIP")
    else:
        print(f"📁 Uploaded {filename}")


**Option B: Use Google Drive**

If your dataset is in Google Drive, mount it and copy the data.


In [ ]:
# Mount Google Drive and copy data
# Uncomment and modify the path to your dataset in Drive
# drive.mount('/content/drive')
# !cp -r '/content/drive/MyDrive/your_dataset_path/*' /content/data/
# print("✅ Data copied from Google Drive!")


## Step 4: Verify Setup

Check that all required files and data are in place.


In [ ]:
# Verify source files
required_files = ['models.py', 'train.py', 'utils.py']
missing_files = []

for file in required_files:
    if os.path.exists(f'src/{file}'):
        print(f"✅ {file} found")
    else:
        print(f"❌ {file} missing")
        missing_files.append(file)

# Verify data structure
if os.path.exists('data/genuine') and os.path.exists('data/forged'):
    genuine_count = len([f for f in os.listdir('data/genuine') if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    forged_count = len([f for f in os.listdir('data/forged') if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    print(f"\n✅ Dataset structure correct")
    print(f"   Genuine signatures: {genuine_count}")
    print(f"   Forged signatures: {forged_count}")
else:
    print("\n❌ Dataset structure incorrect. Please ensure data/genuine/ and data/forged/ exist")

if not missing_files:
    print("\n🎉 Setup complete! Ready to train models.")
else:
    print(f"\n⚠️ Please upload missing files: {missing_files}")


## Step 5: Train Siamese Network with Contrastive Loss

Train the Siamese Network model using Contrastive Loss for signature verification.


In [ ]:
# Import required modules
from train import train_siamese_model
import sys

# Add src to path
sys.path.insert(0, '/content/src')

# Training parameters
data_directory = "data"
epochs = 30
batch_size = 8
learning_rate = 0.001
img_size = (128, 128)

print("=" * 60)
print("SIAMESE NETWORK TRAINING")
print("=" * 60)
print(f"Data directory: {data_directory}")
print(f"Epochs: {epochs}")
print(f"Batch size: {batch_size}")
print(f"Learning rate: {learning_rate}")
print(f"Loss function: Contrastive Loss (margin=1.0, threshold=0.5)")
print("=" * 60)

# Train model
siamese_model, siamese_history, siamese_metrics = train_siamese_model(
    data_directory,
    model_save_path='models/siamese_model.h5',
    epochs=epochs,
    batch_size=batch_size,
    learning_rate=learning_rate,
    img_size=img_size
)

print("\n" + "=" * 60)
print("TRAINING COMPLETED!")
print("=" * 60)
print(f"Model saved to: models/siamese_model.h5")
print(f"Embedding network saved to: models/siamese_embedding.h5")
print(f"\nFinal Metrics:")
print(f"  Accuracy: {siamese_metrics['accuracy']:.4f}")
print(f"  Precision: {siamese_metrics['precision']:.4f}")
print(f"  Recall: {siamese_metrics['recall']:.4f}")
print(f"  F1-Score: {siamese_metrics['f1_score']:.4f}")
print("=" * 60)


In [ ]:
# Display training history plot
from IPython.display import Image, display

if os.path.exists('results/siamese_training_history.png'):
    display(Image('results/siamese_training_history.png'))
else:
    print("Training history plot not found")

# Display confusion matrix
if os.path.exists('results/siamese_confusion_matrix.png'):
    display(Image('results/siamese_confusion_matrix.png'))
else:
    print("Confusion matrix not found")


## Step 7: Download Models and Results

Download trained models and results to your local machine.


In [ ]:
# Download trained models
from google.colab import files

# Download Siamese model
if os.path.exists('models/siamese_model.h5'):
    files.download('models/siamese_model.h5')
    print("✅ Downloaded siamese_model.h5")

# Download embedding network
if os.path.exists('models/siamese_embedding.h5'):
    files.download('models/siamese_embedding.h5')
    print("✅ Downloaded siamese_embedding.h5")

# Download results
if os.path.exists('results/siamese_metrics.json'):
    files.download('results/siamese_metrics.json')
    print("✅ Downloaded siamese_metrics.json")

if os.path.exists('results/siamese_confusion_matrix.png'):
    files.download('results/siamese_confusion_matrix.png')
    print("✅ Downloaded siamese_confusion_matrix.png")

if os.path.exists('results/siamese_training_history.png'):
    files.download('results/siamese_training_history.png')
    print("✅ Downloaded siamese_training_history.png")


## Step 8: Test Model Inference (Optional)

Test the trained model on sample images.


In [ ]:
# Load trained model and test on sample images
from tensorflow import keras
from models import create_siamese_network_with_contrastive, compile_siamese_with_contrastive
from utils import prepare_dataset
import numpy as np

# Load model
print("Loading trained model...")
siamese_model = keras.models.load_model('models/siamese_model.h5')
embedding_model = keras.models.load_model('models/siamese_embedding.h5')

# Load test data
print("Loading test data...")
X, y = prepare_dataset('data', (128, 128))
X_test = X[-100:]  # Use last 100 images for testing
y_test = y[-100:]

# Test on a few pairs
from models import prepare_siamese_pairs
test_pairs_a, test_pairs_b, test_labels = prepare_siamese_pairs(X_test, y_test, num_pairs=10)

# Predict
distances = siamese_model.predict([test_pairs_a, test_pairs_b], verbose=0)
predictions = (distances < 0.5).astype(int).flatten()

print("\nSample Predictions:")
print("=" * 60)
for i in range(min(10, len(predictions))):
    actual = "Same" if test_labels[i] == 1 else "Different"
    predicted = "Same" if predictions[i] == 1 else "Different"
    distance = distances[i][0]
    correct = "✅" if test_labels[i] == predictions[i] else "❌"
    print(f"Pair {i+1}: Distance={distance:.4f}, Actual={actual}, Predicted={predicted} {correct}")
print("=" * 60)
